Steps
* calculate size of electrode areas 4680 vs 280Ah vs 500Ah 
* determine number of windings / sheets 
* calculate number of current collector tabs 
* cell volume utilization ratio (electode-seperator-current_collector volume / cell can volume) --> why?
* resistance / impedance to calculate Joule Heating IR² (only valid for external short circuit, not internal)


In [22]:
import sys
sys.path.insert(0, '/home/matthias/Documents/GitHub/Cell_model_energy_costs')
import cell_model as Cellmodel
import pandas as pd

## Making the cells

In [2]:
# Active materials
NMC_cathode = Cellmodel.Activematerial_cathode("NMC811" , 195, 3.86)
LFP_cathode = Cellmodel.Activematerial_cathode("LFP", 155, 3.3)
Gr = Cellmodel.Activematerial_anode("Gr", 344 , 0.17)

# Currentcollectors 
Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)

# Electrolyte 
LP40_standard = Cellmodel.Electrolyte_liquid("LiPF6" , "EC:DMC",
                                      1.1, # concentration mol / L
                                      151.9 # MolarMass LiPF6
                                      )

# Separator
Separator_standard = Cellmodel.Separator("PP+Al" , 12e-4 , 0.44 , 1.18)

# Electrods 
positive_NMC = Cellmodel.Electrodecomposition_cathode_opt1(NMC_cathode.name,
                                                       5.4, # areal capacity
                                                       0.95, # active frac
                                                       3.4, # density
                                                       NMC_cathode
                                                      ) # NMC = 3.4 / LFP = 2.5
positive_LFP = Cellmodel.Electrodecomposition_cathode_opt1(LFP_cathode.name, 
                                                        3.1, 
                                                        0.92, 
                                                        2.5, 
                                                        LFP_cathode
                                                        ) # NMC = 3.4 / LFP = 2.5
negative_NMC = Cellmodel.Electrodecomposition_anode_opt1( Gr.name , positive_NMC.areal_cap*1.1 , 0.965 , 1.6, Gr)
negative_LFP = Cellmodel.Electrodecomposition_anode_opt1( Gr.name , positive_LFP.areal_cap*1.1 , 0.965 , 1.6, Gr)

* The NMC cathode needs to be quite thick, so 5 mAh / cm2 gives about 80 micrometer

### Cylindrical

In [3]:
NMC_Gr_cyl = Cellmodel.Cylindrical("NMC-Gr_Cyl", 
                               positive_NMC, 
                               negative_NMC, 
                               Separator_standard , 
                               LP40_standard,
                               1.7, # electrolyte content mL/mAh
                               4.6, # diameter
                               8.0, # height 
                               0.06, # can thickness
                               7.9, # can density
                               0.3, # void diameter
                               0.6, # head space
                               4, # extra mass
                               0.94, # LLI
                               NMC_cathode, 
                               Gr, 
                               Al, 
                               Cu)

In [49]:
cylindrical_surf_area = 2 * 3.14 * (NMC_Gr_cyl.diameter/2) * NMC_Gr_cyl.height \
                        + 2 * (3.14 * (NMC_Gr_cyl.diameter/2)**2)
cylindrical_surf_area

148.77319999999997

In [4]:
NMC_Gr_cyl.jr_area

2488.308195845172

In [5]:
NMC_Gr_cyl.turns

46.37715059996704

In [6]:
positive_NMC.thickness_el_c

0.008573469873779471

In [7]:
NMC_Gr_cyl.energy

93.21421472757248

In [8]:
NMC_Gr_cyl.capacity

25.261304804220185

In [9]:
2376/NMC_Gr_cyl.jr_area

0.9548656408266879

Seems to fit well to the Tesla 4680 NMC811 cell https://insideevs.com/news/598656/tesla-4680-battery-cell-specs/ 

### Prismatic 280 Ah

In [10]:
LFP_Gr_prismatic = Cellmodel.Prismatic("LFP-Gr_Pris", 
                                       positive_LFP, 
                                       negative_LFP, 
                                       Separator_standard, 
                                       LP40_standard, 
                                       1.7, # electrolyte ratio mL / mAh
                                       17, # height
                                       20, # width 
                                       7, # depth
                                       0.5, # can thickness
                                       7.9, # can density
                                       1.5, # terminal clearance
                                       1, # nrolls
                                       40, # extramass
                                       0.94, # LLI
                                       LFP_cathode, 
                                       Gr, 
                                       Al, 
                                       Cu)

Assuming only 1 jelly roll in the prismatic can

In [55]:
prismatic_surf_area = 2 * (LFP_Gr_prismatic.length * LFP_Gr_prismatic.width) \
                        + 2 * (LFP_Gr_prismatic.length * LFP_Gr_prismatic.height) \
                        + 2 * (LFP_Gr_prismatic.width * LFP_Gr_prismatic. height) 
prismatic_surf_area

59768.66804692929

In [11]:
LFP_Gr_prismatic.jr_area

49298.30046928924

In [12]:
LFP_Gr_prismatic.turns

85.42171542648776

In [13]:
positive_LFP.thickness_el_c

0.008695652173913044

In [14]:
LFP_Gr_prismatic.energy

899.2818497726053

In [15]:
LFP_Gr_prismatic.capacity

287.3104951350177

Seems to fit to the CATL 280 Ah cell 
* https://www.evlithium.com/CATL-Battery-Cell/catl-lifepo4-battery-280ah.html 
* https://www.catl.com/en/uploads/1/file/public/202205/20220512225901_d0k23qtykm.pdf

### Prismatic 560 Ah

In [16]:
LFP_Gr_prismatic_560 = Cellmodel.Prismatic("LFP-Gr_Pris", 
                                       positive_LFP, 
                                       negative_LFP, 
                                       Separator_standard, 
                                       LP40_standard, 
                                       1.7, # electrolyte ratio mL / mAh
                                       17, # height
                                       20*2, # width 
                                       7, # depth
                                       0.65, # can thickness
                                       7.9, # can density
                                       1.5, # terminal clearance
                                       1, # nrolls
                                       40, # extramass
                                       0.94, # LLI
                                       LFP_cathode, 
                                       Gr, 
                                       Al, 
                                       Cu)

Assuming only 1 jelly roll in the prismatic can

In [56]:
prismatic_surf_area_560 = 2 * (LFP_Gr_prismatic_560.length * LFP_Gr_prismatic_560.width) \
                        + 2 * (LFP_Gr_prismatic_560.length * LFP_Gr_prismatic_560.height) \
                        + 2 * (LFP_Gr_prismatic_560.width * LFP_Gr_prismatic_560. height) 
prismatic_surf_area_560

83441.89853835297

In [17]:
LFP_Gr_prismatic_560.jr_area

97317.69013460062

In [18]:
LFP_Gr_prismatic_560.turns

81.11529424432618

In [19]:
LFP_Gr_prismatic_560.energy

1775.2342690669361

In [20]:
LFP_Gr_prismatic_560.capacity

567.1674981044524

* https://www.takomabattery.com/eve-releases-560ah-large-capacity-energy-storage-battery-lf560k/


## Combining info

In [57]:
energy_pack = 100000 # Wh
df_all = pd.DataFrame(index = ['Jelly Roll Area / cm2', 'Turns / #', 
                               'Capacity / Ah', 'Energy / Wh', 
                               'Cells@100kWh / #', 
                              ' Surface Area / cm2'])
df_all['Cylindric 4680'] = [NMC_Gr_cyl.jr_area, 
                 NMC_Gr_cyl.turns, 
                 NMC_Gr_cyl.capacity,
                 NMC_Gr_cyl.energy,
                 energy_pack/NMC_Gr_cyl.energy, 
                cylindrical_surf_area]
df_all['Prismatic 280Ah'] = [LFP_Gr_prismatic.jr_area,
                            LFP_Gr_prismatic.turns,
                            LFP_Gr_prismatic.capacity,
                            LFP_Gr_prismatic.energy,
                            energy_pack/LFP_Gr_prismatic.energy,
                            prismatic_surf_area]
df_all['Prismatic 560Ah'] = [LFP_Gr_prismatic_560.jr_area,
                            LFP_Gr_prismatic_560.turns,
                            LFP_Gr_prismatic_560.capacity,
                            LFP_Gr_prismatic_560.energy,
                            energy_pack/LFP_Gr_prismatic_560.energy,
                            prismatic_surf_area_560]
df_all.round(decimals=1)

Cylindric 4680  Prismatic 280Ah  Prismatic 560Ah
Jelly Roll Area / cm2          2488.3          49298.3          97317.7
Turns / #                        46.4             85.4             81.1
Capacity / Ah                    25.3            287.3            567.2
Energy / Wh                      93.2            899.3           1775.2
Cells@100kWh / #               1072.8            111.2             56.3
 Surface Area / cm2             148.8          59768.7          83441.9